In [1]:
import loompy
import pandas as pd
import numpy as np
import scipy.sparse as sp
import pickle

/home/ppolicar/nfs/miniconda/envs/tsne/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ppolicar/nfs/miniconda/envs/tsne/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data = loompy.connect('../../data/L5_All.loom')

In [3]:
counts = data.sparse().tocsr(copy=False)

In [4]:
counts

<27998x160796 sparse matrix of type '<class 'numpy.float32'>'
	with 253062766 stored elements in Compressed Sparse Row format>

In [5]:
genes = pd.Index(data.ra.Gene).str.upper()

In [6]:
duplicated_gene_mask = genes.duplicated()
genes = genes[~duplicated_gene_mask].values
counts = counts[~duplicated_gene_mask]

In [7]:
def calculate_cpm(x, axis=1):
    normalization = np.sum(x, axis=axis)
    # On sparse matrices, the sum will be 2d. We want a 1d array
    normalization = np.squeeze(np.asarray(normalization))
    # Straight up division is not an option since this will form a full dense
    # matrix if `x` is sparse. Divison can be expressed as the dot product with
    # a reciprocal diagonal matrix
    normalization = sp.diags(1 / normalization, offsets=0)
    if axis == 0:
        cpm_counts = np.dot(x, normalization)
    elif axis == 1:
        cpm_counts = np.dot(normalization, x)
    return cpm_counts * 1e6


def log_normalize(data):
    if sp.issparse(data):
        data = data.copy()
        data.data = np.log2(data.data + 1)
        return data

    return np.log2(data.astype(np.float64) + 1)

In [8]:
cpm_counts = calculate_cpm(counts, axis=0)
log_counts = log_normalize(cpm_counts)

In [9]:
loompy.create(
    'data/zeisel_2018.loom',
    {'': counts, 'log_counts': log_counts},
    row_attrs={'Gene': genes},
    col_attrs={'CellID': data.ca.CellID,
               'CellType1': data.ca.TaxonomyRank3,
               'CellType2': data.ca.TaxonomyRank4}
)

In [10]:
data_dict = {'counts': counts,
             'log_counts': log_counts,
             'Gene': genes,
             'CellID': data.ca.CellID,
             'CellType1': data.ca.TaxonomyRank3,
             'CellType2': data.ca.TaxonomyRank4}

In [11]:
with open('data/zeisel_2018.pickle', 'wb') as f:
    pickle.dump(data_dict, f)